Name TBD

In [118]:
import scipy.io as sio
import numpy as np
import os
import re
from matplotlib import pyplot as plt

In [119]:
#########################################################################################
###########################  Defining variables
#########################################################################################

i = 0
path = '/Users/joseferpaz/Documents/Vanderbilt Internship/fw_i3cm1i_3pluspoint_berglund_QPBO/test_cases'
file_list = os.listdir(path)    #This is a list of all of the .mat files
vtk_max = 2**14-1


### temporary AlgoParams substitute
decoupled_estimation = True # flag for decoupled R2 estimation
Fibonacci_search = True # flag for Fibonacci search
B0_smooth_in_stack_direction = False # flag for B0 smooth in stack direction
multigrid = True # flag for multi-level resolution pyramid
estimate_R2 = True # flag to estimate R2star"""
verbose = True# flag for verbose status messages (default false)
R2_stepsize = 1
mu = 10
max_R2 = 120; # maximum R2 in s^-1
max_label_change = 0.1; #
fine_R2_stepsize = 1.0; # Fine stepsize for discretization of R2(*) [s^-1] (used in decoupled fine-tuning step)
coarse_R2_stepsize = 10.0; # Coarse stepsize for discretization of R2(*) [s^-1] (used for joint estimation step, value 0.0 --> Decoupled estimation only
water_R2 = 0.0; # Water R2 [sec-1]


DixonApp_parent_folder = os.path.abspath('/Users/joseferpaz/Documents/Vanderbilt Internship/fw_i3cm1i_3pluspoint_berglund_QPBO/berglund/QPBO/DixonApp')
DixonApp_vtk_folder = DixonApp_parent_folder + "/vtk"

print DixonApp_vtk_folder

DixonApp_vtk_prefix_input  = 'DixonApp_Input'
DixonApp_vtk_prefix_output = 'DixonApp_Output'

edit_path = path + '/' + file_list[i]
header = """
    # vtk DataFile Version 3.0);
    created by writeVTK (Matlab implementation by Erik Vidholm)
    BINARY
    DATASET STRUCTURED_POINTS
    """

/Users/joseferpaz/Documents/Vanderbilt Internship/fw_i3cm1i_3pluspoint_berglund_QPBO/berglund/QPBO/DixonApp/vtk


In [151]:
#########################################################################################
############## writing VTK files
#########################################################################################

def write_VTK_simple(vol,vtkfile):
    
    print vtkfile
    print vol.shape
    print vol.dtype
    
    # eliminate echo dimension
    vol = vol.squeeze()
    
    # byteswap when needed
    vol = vol.byteswap()
    
    # make slice dimension the first index
    vol = vol.transpose( (2, 0, 1) )

    vol.astype('int16').tofile(vtkfile)
    
    with open(vtkfile, 'r+') as f:
        content = f.read()
        f.seek(0, 0)
        f.write(header.rstrip('\r\n') + '\n' + content)
    
    #plt.imshow(vol[:,:,0,0], interpolation='nearest', cmap='gray')
    #plt.show()

def write_VTK(vol,vtkfile):

    #where vol is the 3D matrix... but is it really?
    #it isn't but we ignore the rest because we don't need it
    sz = np.shape(vol)
    X = sz[0]
    Y = sz[1]
    Z = 1

    if (len(sz) == 3):
        Z = sz[2]

    header = """# vtk DataFile Version 3.0);
created by writeVTK (Matlab implementation by Erik Vidholm... sort of)
BINARY
DATASET STRUCTURED_POINTS
"""
    
    header += '%s%d%c%d%c%d\n' % ('DIMENSIONS ', X, ' ', Y, ' ', Z);
    header += '%s%f%c%f%c%f\n' % ('ORIGIN ', 0.0, ' ', 0.0, ' ', 0.0);
    header += '%s%f%c%f%c%f\n' % ('SPACING ', 1.0, ' ', 1.0, ' ', 1.0);
    header += '%s%d\n' % ('POINT_DATA ', X*Y*Z);

    vol_dataType = vol.dtype

    if vol_dataType == 'uint8':
        header += 'SCALARS image_data unsigned_char\n'
    elif vol_dataType == 'int8':
        header += 'SCALARS image_data char\n'
    elif vol_dataType == 'uint16':
        header += 'SCALARS image_data unsigned_short\n'
    elif vol_dataType == 'int16':
        header += 'SCALARS image_data short\n'
    elif vol_dataType == 'uint32':
        header += 'SCALARS image_data unsigned_int\n'
    elif vol_dataType == 'int32':
        header += 'SCALARS image_data int\n'
    elif vol_dataType == 'float32':
        header += 'SCALARS image_data float\n'
    elif vol_dataType == 'float64':
        header += 'SCALARS image_data double\n'

    header += 'LOOKUP_TABLE default\n' #Make sure to print 'header' to check
    
    # eliminate echo dimension
    vol = vol.squeeze()
    
    # byteswap when needed
    vol = vol.byteswap()
    
    # make slice dimension the first index
    vol = vol.transpose( (2, 0, 1) )
    
    # write the data into the file
    vol.astype('int16').tofile(vtkfile)
    
    #If writting files to check binary data, just comment the following 4 lines of code
    with open(vtkfile, 'r+') as f:
        content = f.read()
        f.seek(0, 0)
        f.write(header.rstrip('\r\n') + '\n' + content)
    
    #plt.imshow(vol[:,:,0,0], interpolation='nearest', cmap='gray')
    #plt.show()
    


In [152]:
def read_VTK(vtkfile):

    fid = open(vtkfile, "r")

    fid.readline()  # vtk DataFile Version x.x
    fid.readline() # comments
    fid.readline() # BINARY
    fid.readline() # DATASET STRUCTURED_POINTS

    z = fid.readline() # DIMENSIONS NX NY NZ   #5

    #sz = sscanf(s, '%*s%d%d%d')
    sz = map(int, re.findall(r'\d+', z))    #5

    ###THE EXAMPLE TEST WAS READING LINES JUST Fine
    ### KEEP AN EYE ON THE BINARY stuffVC


    fid.readline() # ORIGIN OX OY OZ
    fid.readline() # SPACING SX SY SZ
    fid.readline() # POINT_DATA NXNYNZ


    s = fid.readline() # SCALARS/VECTORS name data_type (ex: SCALARS imagedata unsigned_char)
    #svstr = sscanf(s, '%s', 1);
    #dtstr = sscanf(s, '%*s%*s%s');
    ##### Not finished - still in MatLab
    

In [147]:
#########################################################################################
############## Calling the information from the mat file
#########################################################################################

mat = sio.loadmat(edit_path)

## In the future this will be imported from another file ##
num_array = mat['imDataParams']['images'] 
echo_times = mat['imDataParams']['TE']

In [148]:
#all of this is supposed to happen for every mat file 
#but worry about that later

image_array = num_array[0][0]

        ##getting dimensions
nx, ny, nz, nc, ne = np.shape(image_array)
print "The dimensions are:" , np.shape(image_array)

print np.max( np.real(image_array) )

        ##get maximum values
max_real = np.amax(abs(np.real(image_array)))
max_imag = np.amax(abs(np.imag(image_array)))

if max_real > max_imag:
    images_max = max_real
else:
    images_max = max_imag

print "This is the MAX of the real # : " , max_real
print "This is the MAX of the imaginary # : " , max_imag
print "This is the MAX out of both # : " , images_max
print "-" * 100
print "These are the echo times: " , echo_times


The dimensions are: (192, 192, 4, 1, 6)
4.69632e+07
This is the MAX of the real # :  4.69632e+07
This is the MAX of the imaginary # :  5.52753e+07
This is the MAX out of both # :  5.52753e+07
----------------------------------------------------------------------------------------------------
These are the echo times:  [[ array([[ 0.001436,  0.003084,  0.004732,  0.00638 ,  0.008028,  0.009676]])]]


In [149]:
#Attempting to get the values that are ultimately written in the file ----NOT FINISHED
for idx_echo in range(ne):

    image_re = np.array(np.real(image_array[:,:,:,:,idx_echo])*(vtk_max/images_max), dtype = np.int16)
    image_im = np.array(np.imag(image_array[:,:,:,:,idx_echo])*(vtk_max/images_max), dtype = np.int16)

    #vtk file names that have to be part of a loop
    vtk_filename_re = '%s/%s_echo%04d_re.vtk' % (DixonApp_vtk_folder, DixonApp_vtk_prefix_input, idx_echo)
    vtk_filename_im = '%s/%s_echo%04d_im.vtk' % (DixonApp_vtk_folder, DixonApp_vtk_prefix_input, idx_echo)

    write_VTK(image_re, vtk_filename_re)
    write_VTK(image_im, vtk_filename_im)

    #Now as part of the loop, write the vtk files!

    ## SO (one for the real and one for the imaginary ) for every iteration
    ###print image_re.shape
    ###print np.max(image_re)
    ###print np.min(image_re)
    ###print vtk_max
    
    ###plt.imshow(image_re[:,:,0,0], interpolation='nearest', cmap='gray')
    ###plt.show()
    
    ###plt.imshow(image_im[:,:,0,0], interpolation='nearest', cmap='gray')
    ###plt.show()



int16


In [150]:
##########################################################################################
########################### Making THE string
##########################################################################################

path_to_DixonApp_executable = DixonApp_parent_folder + "/MAC/DixonApp_MAC.exe"
path_to_DixonApp_libraries = DixonApp_parent_folder + "/MAC/lib"
system_command_string_prefix = 'export DYLD_LIBRARY_PATH="%s"; "%s"' % (path_to_DixonApp_libraries, path_to_DixonApp_executable)

In [112]:
#from here down in the making string section it doesn't work yet
system_command_string_prefix = '%s "%s"' % (system_command_string_prefix, DixonApp_vtk_folder)
system_command_string_prefix = '%s "%s"' % (system_command_string_prefix, DixonApp_vtk_prefix_input)
system_command_string_prefix = '%s "%s"' % (system_command_string_prefix, DixonApp_vtk_prefix_output)


system_command_string = system_command_string_prefix


# flags (6)
system_command_string = '%s "%d"' % (system_command_string, verbose);
system_command_string = '%s "%d"' % (system_command_string, estimate_R2);
system_command_string = '%s "%d"' % (system_command_string, decoupled_estimation);
system_command_string = '%s "%d"' % (system_command_string, Fibonacci_search);
system_command_string = '%s "%d"' % (system_command_string, multigrid);
system_command_string = '%s "%d"' % (system_command_string, B0_smooth_in_stack_direction);

# floats (11)

F = mat['imDataParams']['FieldStrength']; FS = F[0][0]; ## temporary fix
system_command_string = '%s "%.5E"' % (system_command_string, FS);

system_command_string = '%s "%.5e"' % (system_command_string, mu)
system_command_string ='%s "%.5e"' % (system_command_string, R2_stepsize)
system_command_string ='%s "%.5e"' % (system_command_string, max_R2)
system_command_string ='%s "%.5e"' % (system_command_string, max_label_change)
#system_command_string = sprintf('%s "%.5e"', system_command_string, input.InplaneOverThroughplaneVoxelsize);
system_command_string ='%s "%.5e"' % (system_command_string, fine_R2_stepsize)
system_command_string ='%s "%.5e"' % (system_command_string, coarse_R2_stepsize)
system_command_string ='%s "%.5e"' % (system_command_string, water_R2)

print system_command_string



export DYLD_LIBRARY_PATH="/Users/joseferpaz/Documents/Vanderbilt Internship/fw_i3cm1i_3pluspoint_berglund_QPBO/berglund/QPBO/DixonApp/MAC/lib"; "/Users/joseferpaz/Documents/Vanderbilt Internship/fw_i3cm1i_3pluspoint_berglund_QPBO/berglund/QPBO/DixonApp/MAC/DixonApp_MAC.exe" "/Users/joseferpaz/Documents/Vanderbilt Internship/fw_i3cm1i_3pluspoint_berglund_QPBO/berglund/QPBO/DixonApp/vtk" "DixonApp_Input" "DixonApp_Output" "1" "1" "1" "1" "1" "0" "3.00000E+00" "1.00000e+01" "1.00000e+00" "1.20000e+02" "1.00000e-01" "1.00000e+00" "1.00000e+01" "0.00000e+00"
